In [11]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([("word1", "word2"), ("word2","word3")])

#define the conditionnal probability distributions (CPDs)

cpd_word1 = TabularCPD(variable="word1", variable_card=2, values = [[0.6,],[0.4]]) #P(word1)

cpd_word2 = TabularCPD(variable="word2",  variable_card = 2, values = [[0.5,0.4],[0.5,0.6]],
                       evidence=["word1"], evidence_card = [2]) #P(word2 | word1)

cpd_word3 = TabularCPD(variable="word3",  variable_card = 2, values = [[0.7,0.8],[0.3,0.2]],
                       evidence=["word2"], evidence_card = [2]) #P(word3 | word2)

# add the CPDs to the model
model.add_cpds(cpd_word1, cpd_word2, cpd_word3)

# Check the model fpr consistency

assert model.check_model()

# Perform inference
inference = VariableElimination(model)
result = inference.query(variables = ["word3"], evidence = {"word1" :1, "word2" : 0})

print(result)


+----------+--------------+
| word3    |   phi(word3) |
+==========+==============+
| word3(0) |       0.7000 |
+----------+--------------+
| word3(1) |       0.3000 |
+----------+--------------+


In [13]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the model structure
model = DiscreteBayesianNetwork([("w1", "w2"), ("w2", "w3")])

# P(w1): prior over subjects
# w1 = 0 → dog, 1 → cat
cpd_w1 = TabularCPD(
    variable="w1",
    variable_card=2,
    values=[[0.5],  # dog
            [0.5]]  # cat
)

# P(w2 | w1): verbs depend on subjects
# w2 = 0 → barks, 1 → runs
cpd_w2 = TabularCPD(
    variable="w2",
    variable_card=2,
    values=[[0.7, 0.0],   # P(barks | dog)=0.7 ; P(barks | cat)=0.0
            [0.3, 1.0]],  # P(runs | dog)=0.3 ; P(runs | cat)=1.0
    evidence=["w1"],
    evidence_card=[2]
)

# P(w3 | w2): adverbs depend on verbs
# w3 = 0 → loudly, 1 → quickly
cpd_w3 = TabularCPD(
    variable="w3",
    variable_card=2,
    values=[[0.8, 0.1],   # P(loudly | barks)=0.8 ; P(loudly | runs)=0.1
            [0.2, 0.9]],  # P(quickly | barks)=0.2 ; P(quickly | runs)=0.9
    evidence=["w2"],
    evidence_card=[2]
)

# Add CPDs to model
model.add_cpds(cpd_w1, cpd_w2, cpd_w3)
assert model.check_model()

# Inference
inference = VariableElimination(model)

# Example 1: what is the probability distribution of adverbs if we know the subject is "dog"?
result_dog = inference.query(variables=["w3"], evidence={"w1": 0})
print("Adverb probabilities given 'dog':")
print(result_dog)

#  Example 2: same but for "cat"
result_cat = inference.query(variables=["w3"], evidence={"w1": 1})
print("\nAdverb probabilities given 'cat':")
print(result_cat)


Adverb probabilities given 'dog':
+-------+-----------+
| w3    |   phi(w3) |
+=======+===========+
| w3(0) |    0.5900 |
+-------+-----------+
| w3(1) |    0.4100 |
+-------+-----------+

Adverb probabilities given 'cat':
+-------+-----------+
| w3    |   phi(w3) |
+=======+===========+
| w3(0) |    0.1000 |
+-------+-----------+
| w3(1) |    0.9000 |
+-------+-----------+
